In [11]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input


In [12]:

util.startLoop()

In [13]:
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

<IB connected to 127.0.0.1:7497 clientId=1>

In [14]:
# Definir el contrato para el futuro del crudo ligero (CL) Mayo 2025 en NYMEX
contract = Future('CL', '202505', 'NYMEX')

In [15]:
# Establecer el rango de tiempo
end_time = datetime.now()
start_time = end_time - timedelta(days=90)

In [16]:
# Obtener datos históricos
bars = ib.reqHistoricalData(
    contract,
    endDateTime=end_time,
    durationStr='90 D',
    barSizeSetting='1 hour',
    whatToShow='MIDPOINT',
    useRTH=False
)


In [17]:
# Convertir los datos a DataFrame
df = util.df(bars)

In [18]:
# Obtener noticias relevantes sobre el petróleo crudo
API_KEY = 'b35c56d955ee45178c703f7f79c1dfca'
news_url = f'https://newsapi.org/v2/everything?q=crude%20oil%20OR%20NYMEX%20CL&apiKey={API_KEY}'
response = requests.get(news_url)
news_data = response.json()

In [19]:
titles = [article['title'] for article in news_data['articles']]
print(f"Titulares de noticias relevantes:\n {titles}")

Titulares de noticias relevantes:
 ["Gas prices are headed higher, and a 'ripple effect' could make it worse in certain regions", 'Oil slips to lowest level in 6 months amid trade war fears', 'Crude Oil Rebounds Amid Apparent Short-Covering Rally, but Will Bulls Return?', 'Top Oil Stocks To Watch Now – March 7th', 'TESTED: Industry Nine Hydra2 Hub', 'Leishmaniasis in deployed military populations: A systematic review and meta-analysis', 'LC-MS analysis and antioxidant, antibacterial, and antidiabetic activity of Jumli Marshi rice from Nepal: An in vitro and in silico investigation to validate their potential as a functional food', 'Healthcare utilization trends in adults with asthma or COPD during the first year of COVID-19 pandemic in comparison to pre-pandemic: A population-based study', 'Acute and sub-acute toxicity of ethanol extracts of Hagenia abyssinica and Rumex abyssinicus flowers in Swiss albino mice', 'Predictors of clozapine concentration and psychiatric symptoms in patient

In [20]:
# Análisis de sentimiento de noticias
positive_keywords = ['rise', 'growth', 'record', 'profit']
negative_keywords = ['fall', 'loss', 'decline', 'drop']


In [21]:
positive_news = sum(1 for title in titles if any(word in title.lower() for word in positive_keywords))
negative_news = sum(1 for title in titles if any(word in title.lower() for word in negative_keywords))
impact_factor = 1 + (positive_news - negative_news) * 0.001


In [22]:
# Seleccionar datos y normalizar
data = df[['open', 'high', 'low', 'close']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
scaled_data_adjusted = scaled_data * impact_factor

In [23]:
# Identificar soportes y resistencias
def find_support_resistance(data):
    levels = []
    for i in range(2, len(data) - 2):
        if data[i] < data[i - 1] and data[i] < data[i + 1]:
            levels.append((i, data[i]))  # Soporte
        elif data[i] > data[i - 1] and data[i] > data[i + 1]:
            levels.append((i, data[i]))  # Resistencia
    return levels

support_resistance_levels = find_support_resistance(df['close'].values)
print(f"Niveles de soporte y resistencia detectados: {support_resistance_levels}")

Niveles de soporte y resistencia detectados: [(2, np.float64(69.06)), (3, np.float64(69.09)), (5, np.float64(68.94)), (7, np.float64(69.31)), (8, np.float64(69.28)), (9, np.float64(69.35)), (11, np.float64(68.28)), (12, np.float64(68.34)), (14, np.float64(67.77)), (15, np.float64(67.83)), (17, np.float64(67.29)), (21, np.float64(67.59)), (22, np.float64(67.53)), (24, np.float64(67.6)), (26, np.float64(67.34)), (28, np.float64(67.39)), (29, np.float64(67.24)), (33, np.float64(67.83)), (34, np.float64(67.75)), (37, np.float64(68.09)), (39, np.float64(67.4)), (41, np.float64(67.65)), (42, np.float64(67.32)), (43, np.float64(67.49)), (45, np.float64(67.22)), (46, np.float64(67.32)), (47, np.float64(67.28)), (49, np.float64(67.62)), (50, np.float64(67.52)), (51, np.float64(67.53)), (52, np.float64(67.48)), (53, np.float64(67.5)), (54, np.float64(67.46)), (57, np.float64(67.92)), (61, np.float64(66.29)), (65, np.float64(67.72)), (71, np.float64(67.41)), (73, np.float64(67.13)), (74, np.float

In [24]:
# Crear secuencias de datos para LSTM
sequence_length = 60
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data_adjusted)):
    x_train.append(scaled_data_adjusted[i-sequence_length:i])
    y_train.append(scaled_data_adjusted[i])

x_train, y_train = np.array(x_train), np.array(y_train)


In [25]:
# Definir modelo LSTM
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=4))

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0857
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0079
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0070
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0059
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0054
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0050
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0048
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0041
Epoch 9/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0043
Epoch 10/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0042
Epoch 11/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.0039
Epoch 12/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0037
Epoch 13/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0035
Epoch 14/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0032
Epoch 15/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0029
Epoc

In [26]:
# Predicciones
recent_data = scaled_data_adjusted[-sequence_length:]
predictions = []
for _ in range(10):
    x_input = np.array([recent_data])
    predicted_price = model.predict(x_input)
    predictions.append(predicted_price[0])
    recent_data = np.append(recent_data[1:], [predicted_price[0]], axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [27]:
# Desnormalizar predicciones
predictions = scaler.inverse_transform(predictions)
prediction_times = [end_time.replace(hour=17, minute=0, second=0) + timedelta(hours=i) for i in range(1, 11)]
predicted_df = pd.DataFrame(predictions, columns=['open', 'high', 'low', 'close'], index=prediction_times)
print(predicted_df)


                                 open       high        low      close
2025-03-17 18:00:00.627688  67.303579  67.535075  67.210417  67.420772
2025-03-17 19:00:00.627688  67.285167  67.516910  67.193854  67.401578
2025-03-17 20:00:00.627688  67.283221  67.514966  67.193340  67.399343
2025-03-17 21:00:00.627688  67.291242  67.522783  67.201930  67.407506
2025-03-17 22:00:00.627688  67.305336  67.536536  67.215810  67.422019
2025-03-17 23:00:00.627688  67.323182  67.553974  67.232903  67.440415
2025-03-18 00:00:00.627688  67.343397  67.573761  67.252080  67.461205
2025-03-18 01:00:00.627688  67.365144  67.595098  67.272713  67.483486
2025-03-18 02:00:00.627688  67.387913  67.617492  67.294424  67.506706
2025-03-18 03:00:00.627688  67.411382  67.640627  67.316967  67.530521
